<a href="https://colab.research.google.com/github/RyuMinHo/GAI_project/blob/main/image_gradio_gui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio vllm transformers triton PyMuPDF Pillow sentence_transformers numpy typing open_clip_torch
!python -m spacy download en_core_web_sm

Exception in thread Thread-5 (attachment_entry):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 237, in listen
    sock, _ = endpoints_listener.accept()
  File "/usr/lib/python3.10/socket.py", line 293, in accept
    fd, addr = self._accept()
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy.py", line 52, in attachment_entry
    debugpy.listen(_dap_port)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/public_api.py", line 31, in wrapper
    return wrapped(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 143, in debug
    log.reraise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 95.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:

import torch
from vllm import LLM
import spacy
from sentence_transformers import SentenceTransformer
from vllm.sampling_params import SamplingParams
import open_clip
from PIL import Image
import torch.nn.functional as F

def init_models():
    try:
        print("LLaVA 모델 로딩 중...")
        llm = LLM(model="llava-hf/llava-v1.6-mistral-7b-hf",
                  dtype='half',
                  max_model_len=8192,
                  device="cuda" if torch.cuda.is_available() else "cpu")

        print("CLIP 모델 로딩 중...")
        model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
        tokenizer = open_clip.get_tokenizer('ViT-B-32')

        print("SpaCy 모델 로딩 중...")
        nlp = spacy.load("en_core_web_sm")

        print("SentenceTransformer 로딩 중...")
        embedder = SentenceTransformer('all-MiniLM-L6-v2')

        print("모든 모델 초기화 완료!")
        return llm, model, preprocess, tokenizer, nlp, embedder
    except Exception as e:
        print(f"모델 초기화 중 오류 발생: {str(e)}")
        return None, None, None, None, None, None

# 전역 변수로 모델 초기화
print("모델 초기화 시작...")
GLOBAL_LLM, GLOBAL_CLIP_MODEL, GLOBAL_CLIP_PREPROCESS, GLOBAL_CLIP_TOKENIZER, GLOBAL_NLP, GLOBAL_EMBEDDER = init_models()
print("사용 가능한 모델:", "LLaVA" if GLOBAL_LLM else "None")

모델 초기화 시작...
LLaVA 모델 로딩 중...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WARNING 11-27 08:13:54 config.py:1865] Casting torch.bfloat16 to torch.float16.
INFO 11-27 08:14:03 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
INFO 11-27 08:14:03 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='llava-hf/llava-v1.6-mistral-7b-hf', speculative_config=None, tokenizer='llava-hf/llava-v1.6-mistral-7b-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=F

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 11-27 08:14:10 model_runner.py:1077] Loading model weights took 14.0785 GB
INFO 11-27 08:14:12 worker.py:232] Memory profiling results: total_gpu_memory=39.56GiB initial_memory_usage=14.62GiB peak_torch_memory=14.94GiB memory_usage_post_profile=14.66GiB non_torch_memory=0.57GiB kv_cache_size=20.09GiB gpu_memory_utilization=0.90
INFO 11-27 08:14:12 gpu_executor.py:113] # GPU blocks: 10288, # CPU blocks: 2048
INFO 11-27 08:14:12 gpu_executor.py:117] Maximum concurrency for 8192 tokens per request: 20.09x
INFO 11-27 08:14:15 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-27 08:14:15 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
I

In [ ]:
import gradio as gr
import base64
import os
from PIL import Image
import io
import torch
from vllm.sampling_params import SamplingParams

def detect_image_type(image):
    """OpenCLIP을 사용한 이미지 타입 감지 함수"""
    try:
        image_input = GLOBAL_CLIP_PREPROCESS(image).unsqueeze(0)

        categories = [
            "a photo of a car or vehicle",
            "a photo of nature or landscape",
            "a financial document or spreadsheet",
            "a medical pill or medication",
            "a photo of an animal",
            "a general photograph"
        ]

        text = GLOBAL_CLIP_TOKENIZER(categories)

        with torch.no_grad():
            image_features = GLOBAL_CLIP_MODEL.encode_image(image_input)
            text_features = GLOBAL_CLIP_MODEL.encode_text(text)

            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)

            similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            values, indices = similarity[0].topk(1)

        category_mapping = {
            0: "car",
            1: "nature",
            2: "financial",
            3: "pill",
            4: "animal",
            5: "general"
        }

        detected_type = category_mapping[indices[0].item()]
        confidence = values[0].item()

        print(f"Detected type: {detected_type} with confidence: {confidence:.2f}")

        return detected_type

    except Exception as e:
        print(f"이미지 타입 감지 중 오류 발생: {str(e)}")
        return "general"

def get_domain_specific_prompt(image_type, is_korean=True):


    """도메인별 특화된 프롬프트 생성"""
    prompts = {
        "car": {
            "ko": """자동차 이미지를 전문적으로 분석해주세요:
1. 차량 기본 정보
   - 차종 (세단, SUV, 트럭 등)
   - 외관 색상 및 디자인 특징
   - 특징적인 부분 (휠, 그릴 등)

2. 상태 및 환경
   - 차량의 전반적인 상태
   - 촬영된 환경과 각도
   - 특이사항이나 변형 여부""",

            "en": """Please analyze this vehicle image professionally:
1. Basic Vehicle Information
   - Vehicle type (sedan, SUV, truck, etc.)
   - Exterior color and design features
   - Notable parts (wheels, grille, etc.)

2. Condition and Environment
   - Overall vehicle condition
   - Photography environment and angle
   - Any modifications or special features"""
        },

        "nature": {
            "ko": """자연/풍경 이미지를 상세히 분석해주세요:
1. 주요 자연 요소
   - 지형지물 (산, 강, 바다 등)
   - 식물상 (나무, 꽃, 풀 등)
   - 기후/날씨 상태

2. 전체적인 구도
   - 원경/중경/근경의 구성
   - 빛과 그림자의 특징
   - 계절감과 시간대""",

            "en": """Please analyze this nature/landscape image in detail:
1. Key Natural Elements
   - Terrain features (mountains, rivers, ocean, etc.)
   - Flora (trees, flowers, grass, etc.)
   - Climate/weather conditions

2. Overall Composition
   - Background/middle-ground/foreground
   - Light and shadow characteristics
   - Seasonal and temporal aspects"""
        },

        "animal": {
            "ko": """동물 이미지를 매우 상세하고 정확하게 분석해주세요:

1. 종 식별 및 기본 정보
  - 동물의 정확한 종과 품종 (예: 골든 리트리버, 시베리안 허스키 등)
  - 추정 나이대 (새끼, 성견, 노견 등)
  - 개체 수와 성별 특징 (알 수 있는 경우)

2. 외모와 신체적 특징
  - 털색과 피모 상태 (구체적인 색조와 패턴)
  - 체형과 체격 (크기, 골격, 체형 등)
  - 얼굴 특징 (눈 색상, 귀 모양, 코와 입 특징)
  - 특별한 신체적 특징이나 무늬

3. 표정과 감정 상태
  - 각 개체의 구체적인 표정
  - 감정 상태나 기분 (행복해 보임, 졸려 보임 등)
  - 에너지 레벨과 활동성

4. 행동과 상호작용
  - 현재의 자세나 포즈
  - 다른 개체들과의 관계나 상호작용
  - 카메라나 주변 환경과의 반응

5. 환경과 상황
  - 촬영된 장소와 배경
  - 동물의 컨디션과 건강 상태
  - 특별한 상황이나 맥락

주의: 관찰 가능한 특징만을 바탕으로 객관적이고 구체적으로 설명해주세요.
추측이나 불확실한 정보는 제외하고 설명해주세요.""",

    "en": """Please analyze this animal image in great detail:

1. Species Identification & Basic Info
  - Exact species and breed (e.g., Golden Retriever, Siberian Husky)
  - Estimated age group (puppy, adult, senior)
  - Number of individuals and gender characteristics (if apparent)

2. Appearance & Physical Features
  - Fur color and coat condition (specific tones and patterns)
  - Body type and build (size, frame, body shape)
  - Facial features (eye color, ear shape, nose and mouth characteristics)
  - Distinctive physical marks or patterns

3. Expression & Emotional State
  - Specific facial expressions of each individual
  - Emotional state or mood (appears happy, sleepy, etc.)
  - Energy level and activity

4. Behavior & Interaction
  - Current posture or pose
  - Relationship and interaction with other individuals
  - Response to camera or surroundings

5. Environment & Context
  - Location and background of the photo
  - Animal's condition and health appearance
  - Special circumstances or context

Note: Please provide objective and specific descriptions based only on
observable characteristics. Exclude any speculation or uncertain information."""
        },

        "financial": {
            "ko": """재무 문서를 체계적으로 분석해주세요:
1. 문서 유형 확인
   - 재무제표 종류 (재무상태표, 손익계산서 등)
   - 기간 및 날짜 정보
   - 회사/기관명

2. 시각적 구성 요소
   - 표의 구조와 레이아웃
   - 주요 섹션과 카테고리
   - 데이터 표시 형식

주의: 구체적인 숫자나 민감한 재무 정보는 언급하지 않으며,
문서의 구조와 형식적 특징만을 설명합니다.""",

            "en": """Please analyze this financial document systematically:
1. Document Identification
   - Type of financial statement
   - Period and date information
   - Company/Institution name

2. Visual Components
   - Table structure and layout
   - Major sections and categories
   - Data presentation format

Note: Avoid mentioning specific numbers or sensitive financial data,
focus on document structure and formatting characteristics."""
        },

        "pill": {
            "ko": """약품 이미지를 전문적으로 분석해주세요:
1. 약품의 물리적 특성
   - 크기, 모양, 색상
   - 각인 또는 표시된 문자/숫자
   - 특별한 디자인 요소

2. 식별 가능한 정보
   - 약품의 종류나 유형
   - 제조사 표시나 브랜드 요소
   - 단위나 용량 표시

주의: 구체적인 의약품 식별이나 의학적 조언은 하지 않으며,
관찰 가능한 시각적 특징만을 객관적으로 설명해주세요.""",

            "en": """Please analyze this pharmaceutical image professionally:
1. Physical characteristics
   - Size, shape, color
   - Imprints or markings
   - Special design elements

2. Identifiable information
   - Type of medication
   - Manufacturer marks or branding
   - Unit or dosage indicators

Note: Avoid specific drug identification or medical advice,
focus only on observable visual characteristics."""
        }
    }
    return prompts.get(image_type, get_general_prompt(is_korean))

def get_general_prompt(is_korean):
    """일반적인 이미지 분석을 위한 프롬프트 생성"""
    if is_korean:
        return """이미지를 상세히 분석하여 한국어로 설명해주세요:

[사진 유형에 따른 주요 분석 요소]
1. 주요 구성 요소
   - 대상의 종류와 특징
   - 위치와 배치
   - 특징적인 모습이나 상태

2. 장면 구성
   - 전체적인 구도와 분위기
   - 배경과 주변 환경
   - 조명과 색감

3. 세부 사항
   - 특별히 눈에 띄는 요소
   - 독특한 특징이나 패턴
   - 전반적인 상태나 품질

명확하게 보이는 내용만 자연스럽게 통합하여 설명해주세요."""
    else:
        return """Please provide a detailed analysis of the image in English:

[Key Analysis Points]
1. Main Components
   - Type and characteristics of subjects
   - Position and arrangement
   - Distinctive features or conditions

2. Scene Composition
   - Overall layout and atmosphere
   - Background and surroundings
   - Lighting and color scheme

3. Details
   - Notable elements
   - Unique features or patterns
   - Overall condition or quality

Please integrate only clearly visible elements into a natural description."""

def detect_language(text):
    """입력된 텍스트의 언어를 감지하여 한글인지 확인"""
    return any('\u3131' <= char <= '\u318F' or '\uAC00' <= char <= '\uD7A3' for char in text)

def process_message(message, file, mode, history):
    """메시지와 이미지를 처리하는 함수"""
    is_korean = detect_language(message)

    def post_process_response(response):
        """응답 텍스트 후처리"""
        # 중복 문장 제거
        sentences = response.split('.')
        unique_sentences = []
        for s in sentences:
            s = s.strip()
            if s and s not in unique_sentences:
                unique_sentences.append(s)

        # 다시 조합
        clean_response = '. '.join(unique_sentences)
        return clean_response

    if mode == "General Chat":
        # 텍스트만 있는 경우 (일반 채팅)
        if not file:
            try:
                if history and history[-1][1]:  # 이전 대화 컨텍스트가 있는 경우
                    if is_korean:
                        prompt = f"""이전 대화 내용: {history[-1][1]}

새로운 질문: {message}

이전 대화 내용을 참고하여 새로운 질문에 답변해주세요."""
                    else:
                        prompt = f"""Previous context: {history[-1][1]}

New question: {message}

Please answer the new question considering the previous context."""
                else:  # 이전 컨텍스트 없는 일반 대화
                    prompt = message

                sampling_params = SamplingParams(
                    temperature=0.15,    # 조금 더 낮춰서 안정성 확보
                    max_tokens=512,      # 유지
                    top_p=0.85,         # 살짝 낮춰서 더 정확한 어휘 선택
                    repetition_penalty=1.18  # 아주 조금 낮춤
                )

                response = GLOBAL_LLM.generate(
                    prompts=[prompt],
                    sampling_params=sampling_params
                )
                return post_process_response(response[0].outputs[0].text.strip())
            except Exception as e:
                return f"텍스트 처리 중 오류 발생: {str(e)}" if is_korean else f"Error processing text: {str(e)}"

        # 이미지가 있는 경우
        else:
            try:
                image = Image.open(file)
                image_type = detect_image_type(image)
                base_prompt = get_domain_specific_prompt(image_type, is_korean)

                question = message if message.strip() else "이 이미지를 설명해주세요."
                prompt = f"""[INST] <image>
이미지를 보고 설명해주세요:

{base_prompt}

질문: {question}

주의사항:
- 관찰된 내용만 설명
- 한국어로 자연스럽게 작성 [/INST]"""

                # LLaVA 입력 준비
                inputs = {
                    "prompt": prompt,
                    "multi_modal_data": {"image": image},
                }

                sampling_params = SamplingParams(
                    temperature=0.15,    # 조금 더 낮춰서 안정성 확보
                    max_tokens=512,      # 유지
                    top_p=0.85,         # 살짝 낮춰서 더 정확한 어휘 선택
                    repetition_penalty=1.18  # 아주 조금 낮춤
                )

                try:
                    outputs = GLOBAL_LLM.generate(inputs, sampling_params=sampling_params)
                    if outputs and len(outputs) > 0:
                        response = outputs[0].outputs[0].text.strip()
                        if not response:
                            return "응답을 생성하지 못했습니다." if is_korean else "Failed to generate response."
                        return post_process_response(response)
                finally:
                    del inputs
                    if 'outputs' in locals():
                        del outputs
                    import gc
                    gc.collect()
                    torch.cuda.empty_cache()

            except Exception as e:
                print(f"디버그 - 에러 상세: {str(e)}")
                return f"이미지 처리 중 오류 발생: {str(e)}" if is_korean else f"Error processing image: {str(e)}"

    else:  # RAG Chat
        return "RAG 채팅 모드는 아직 구현되지 않았습니다." if is_korean else "RAG chat mode is not implemented yet."

def send_message(message, file, session_name, mode, history):
   if not message.strip() and not file:
       return history, "", None, None

   try:
       # 이미지가 있는 경우
       if file:
           # 이미지와 메시지를 한 쌍으로 히스토리에 추가
           user_message = f"{message}"
           history.append(((file.name, file), user_message))  # 사용자 입력
           response = process_message(message, file, mode, history)
           history.append((None, response))  # 모델 응답
       else:
           # 텍스트만 있는 경우
           history.append((None, message))
           response = process_message(message, file, mode, history)
           history.append((None, response))

       if session_name in state["sessions"]:
           state["sessions"][session_name]["history"] = history

       return history, "", None, None

   except Exception as e:
       print(f"메시지 전송 중 오류 발생: {str(e)}")
       return history, "", None, None

def create_ui():
   custom_css = """
   .message-box {
       display: flex;
       align-items: center;
       gap: 0.5rem;
   }
   .file-btn {
       max-width: 40px;
   }
   .send-btn {
       max-width: 40px;
   }
   .selected-file {
       margin: 0.5rem 0;
       padding: 0.3rem;
       background: #f0f0f0;
       border-radius: 4px;
       font-size: 0.9em;
   }
   """

   state = {
       "sessions": {
           "Example": {"history": [], "mode": "General Chat"}
       },
       "current_session": "Example"
   }

   with gr.Blocks(css=custom_css) as demo:
       with gr.Row():
           # 왼쪽 세션 패널
           with gr.Column(scale=1):
               new_session_btn = gr.Button("+ New Session")
               session_title_input = gr.Textbox(
                   label="Session Title",
                   visible=False
               )
               with gr.Column(elem_classes="session-container"):
                   gr.Markdown("Sessions")
                   session_list = gr.Radio(
                       choices=["Example"],
                       value="Example",
                       interactive=True,
                       label=""
                   )
                   delete_btn = gr.Button("🗑️ Delete Selected Session")

           # 오른쪽 채팅 패널
           with gr.Column(scale=3):
               current_title = gr.Markdown("## Example")

               with gr.Row():
                   chat_mode = gr.Radio(
                       choices=["General Chat", "RAG Chat"],
                       value="General Chat",
                       interactive=True,
                       label=""
                   )

               # 채팅창 레이아웃
               with gr.Column():
                   chatbot = gr.Chatbot(
                       height=400,
                       render_markdown=True,
                       show_copy_button=True,
                       bubble_full_width=False
                   )

                   # 메시지 입력 영역
                   with gr.Row():
                       file_upload = gr.UploadButton(
                           "📎",
                           file_types=["image", "pdf"],
                           scale=1
                       )
                       msg = gr.Textbox(
                           show_label=False,
                           placeholder="Type your message...",
                           container=False,
                           scale=44
                       )
                       send_btn = gr.Button(
                           "↑",
                           scale=1
                       )

                   # Clear Chat 버튼과 Selected File
                   with gr.Row():
                       clear_btn = gr.Button("Clear Chat")
                   with gr.Row():
                       selected_file = gr.Textbox(
                           label="Selected File",
                           interactive=False
                       )

       def show_title_input():
           return gr.update(visible=True)

       def add_session(title):
           if not title or title in state["sessions"]:
               return (
                   gr.update(visible=False),
                   gr.update(choices=list(state["sessions"].keys()), value=state["current_session"])
               )

           new_sessions = {title: {"history": [], "mode": "General Chat"}}
           new_sessions.update(state["sessions"])
           state["sessions"] = new_sessions
           state["current_session"] = title

           choices = list(state["sessions"].keys())

           return (
               gr.update(visible=False),
               gr.update(choices=choices, value=title)
           )

       def switch_session(session_name):
           if session_name in state["sessions"]:
               state["current_session"] = session_name
               return (
                   f"## {session_name}",
                   state["sessions"][session_name]["history"],
                   state["sessions"][session_name]["mode"]
               )
           return current_title, [], chat_mode.value

       def delete_session(session_name):
           if len(state["sessions"]) <= 1:
               gr.Warning("마지막 세션은 삭제할 수 없습니다.")
               return (
                   gr.update(choices=list(state["sessions"].keys()), value=session_name),
                   current_title,
                   chatbot,
                   chat_mode
               )

           if session_name in state["sessions"]:
               del state["sessions"][session_name]
               next_session = next(iter(state["sessions"].keys()))
               state["current_session"] = next_session
               return (
                   gr.update(choices=list(state["sessions"].keys()), value=next_session),
                   f"## {next_session}",
                   state["sessions"][next_session]["history"],
                   state["sessions"][next_session]["mode"]
               )
           return (
               gr.update(choices=list(state["sessions"].keys())),
               current_title,
               chatbot,
               chat_mode
           )

       def chat_mode_change(mode):
           session_name = state["current_session"]
           if session_name in state["sessions"]:
               state["sessions"][session_name]["mode"] = mode
           allowed_types = [".jpg", ".jpeg", ".png"] if mode == "General Chat" else [".pdf"]
           return gr.update(file_types=allowed_types)

       def update_selected_file(file):
           return file.name if file else ""

       def clear_chat():
           session_name = state["current_session"]
           if session_name in state["sessions"]:
               state["sessions"][session_name]["history"] = []
           return [], None

       # Event 연결
       new_session_btn.click(
           show_title_input,
           outputs=session_title_input
       )

       session_title_input.submit(
           add_session,
           inputs=[session_title_input],
           outputs=[session_title_input, session_list]
       )

       session_list.change(
           switch_session,
           inputs=[session_list],
           outputs=[current_title, chatbot, chat_mode]
       )

       delete_btn.click(
           delete_session,
           inputs=[session_list],
           outputs=[session_list, current_title, chatbot, chat_mode]
       )

       chat_mode.change(
           chat_mode_change,
           inputs=[chat_mode],
           outputs=[file_upload]
       )

       file_upload.upload(
           fn=update_selected_file,
           inputs=[file_upload],
           outputs=[selected_file]
       )

       send_btn.click(
           send_message,
           inputs=[msg, file_upload, session_list, chat_mode, chatbot],
           outputs=[chatbot, msg, file_upload, selected_file]
       )

       msg.submit(
           send_message,
           inputs=[msg, file_upload, session_list, chat_mode, chatbot],
           outputs=[chatbot, msg, file_upload, selected_file]
       )

       clear_btn.click(
           clear_chat,
           outputs=[chatbot, selected_file]
       )

   return demo

if __name__ == "__main__":
    demo = create_ui()
    demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:225: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://318db6930872b71271.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
